In [ ]:
import sys
print(sys.executable) #check jupyter kernel
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from data.simu_setting import prox_data
from src.proxITR import proxITR
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler as Scaler
from sklearn.kernel_approximation import Nystroem

### Value Function

In [ ]:
def value_fun(d,A,Y,X,U):
    A = A.to_numpy().reshape(-1)
    Y = Y.to_numpy().reshape(-1)
    U = U.to_numpy().reshape(-1)
    X = X.to_numpy()
    return np.mean((np.abs((np.sign(A-0.5)-d))<0.5)*Y*(1. + np.exp(np.sign(A-0.5)*(0.09375+X @ [0.1875, 0.1875] -0.25*U))))

## Examples for Different Simulation Settings

### Scenario L1

In [ ]:
# Setting
linearity    = 'linear'
Xonly        = 'XW'

In [ ]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

In [ ]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

In [ ]:
# Learn ITRs
rhos = np.power(2.,-np.arange(-2,7))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                                learning_rate= 0.1, n_epoch=2000, opt='LBFGS', verbose=True)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])

d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])

d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos, learning_rate=1e-2)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

In [ ]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X]})

In [ ]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

### Scenario L2

In [ ]:
# Setting
linearity    = 'linear'
Xonly        = 'X'

In [ ]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

In [ ]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

In [ ]:
# Learn ITRs
rhos = np.power(2.,-np.arange(-2,7))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                                learning_rate= 0.1, n_epoch=2000, opt='LBFGS', verbose=True)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])

d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])

d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos, learning_rate=1e-2)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

In [ ]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X]})

In [ ]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

### Scenario N1

In [ ]:
# Setting
linearity    = 'nonlinear'
Xonly        = 'XW'

In [ ]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

In [ ]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

In [ ]:
rhos = np.power(2.,-np.arange(-1,8))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], learning_rate=0.1, n_epoch=2000, batch_size=300, opt='SGD', verbose=False)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])
d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])
d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

In [ ]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X]})

In [ ]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

### Scenario N2

In [ ]:
# Setting
linearity    = 'nonlinear'
Xonly        = 'X'

In [ ]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

In [ ]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

In [ ]:
rhos = np.power(2.,-np.arange(-1,8))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], learning_rate=0.1, n_epoch=2000, batch_size=300, opt='SGD', verbose=False)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])
d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])
d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

In [ ]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X]})

In [ ]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI